<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<span>
<h2>Eliminación de ruido en imagenes de resonancia magnetica, mediante red neuronal convolucional CNN </h2> 
</span>

Procedimiento de entrenamiento con imagenes en formato cerebrales, extraidas previamente de imagenes en formato .NIFTI a JPG

## Importar librerías

In [ ]:
import warnings
warnings.filterwarnings('ignore')
 
import tensorflow 
from tensorflow import keras
from scipy import ndimage, misc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import time
import os
import pandas as pd
import shutil
from pathlib import Path

## Carga y lectura de datos

Se cargarán las imagenes en formato JPG almacenadas en el directorio local
el cual se creo en el Notebook previo 1. Extraccion_Imagenes_Nifti_A_Jpg. Dichas imagenes serán almacenadas en los siguientes conjuntos <b>datos entrenamiento, validación y prueba.</b>

Los datos utilizados en este proyecto fuerón dispuestos por el archivo de imágenes y datos <a href="https://adni.loni.usc.edu/data-samples/access-data/">(IDA)</a> está a cargo del Laboratorio de neuroimagen (LONI) en el Instituto de informática y neuroimagen Mark and Mary Stevens de la USC. Del cual se descargaron 2294 imagenes en formato .nii de 639 pacientes, con un periodo de 3 años de tomas de IMR para el Alzheimer, en personas entre los 55 y 91 años de edad. Dichas imagenes se convirtieron en imagenes en formato .jpg, duplicando la cantidad de las mismas, ya que por cada imagen 3D se extajo la dimension del corte axial y corte coronal, las cuales brindan información valioda para la detección de la enfermedad del Alzheimer, ignorando asi el corte sagital y obteniendo una totalidad de 4588 imagenes en formato JPG.

In [ ]:
"""Categorizacion datos de entrenamiento, validación y prueba

1. Imagenes disponibles en jpg (Corte Axial y Coronal)

clasificacion  cantidad
AD             952
CN             1410
MCI            2226
-----------------------
TOTAL          4588

2. División por cada conjunto de datos

clasificacion  cantidad  entrenamiento (80%)  validacion (19%)  prueba (1%)
AD             952       762                  181               9
CN             1410      1128                 268               14
MCI            2226      1781                 423               22
---------------------------------------------------------------------------
TOTAL          4588      3671                 872               45

"""

df_conjuntos = pd.DataFrame(columns=['clasificacion', 'cantidad' ,'entrenamiento','validacion', 'prueba'])
df_conjuntos.loc[1] = ['AD', 952, 762, 181, 9]  
df_conjuntos.loc[2] = ['CN', 1410, 1128, 268, 14]
df_conjuntos.loc[3] = ['MCI', 2226, 1781, 423, 22]
df_conjuntos

In [ ]:
%run ./Helpers/sqllite_helper.ipynb
helper_sqllite()

In [ ]:
database = r"C:\sqllite\alzheimer_db.db"
conn = create_connection(database)

query = "SELECT * FROM log_metadatos_consolidado"
df_adni = query_table(conn, query)
df_adni

<b>Asignacion de grupos de datos</b>

Clasificacion de conjuntos de datos Entrenamiento, validacion y prueba. Adicional se crean 3 columnas calculadas para almacenar los valores de los directorios donde se clasificaran las imagenes en los sub conjuntos de datos mensionados previamente con el fin de pre procesar las imagenes y por ultimo utilizarlas para contruir el modelo de deteccion de la enfermedad de alzheimer, mediante varias arquitecturas de redes neuronales CNN.

In [ ]:
result_dataframe = pd.DataFrame()
df_conj_iter = df_conjuntos.iterrows()
for index, row in df_conj_iter:
    print(row['clasificacion'], 
          row['entrenamiento'],
          row['validacion'], 
          row['prueba'])
    
    df_actual = df_adni[df_adni['group_m']==row['clasificacion']]
    df_actual = df_actual.reset_index(level=0)
    df_actual['index1'] = df_actual.index
    
    min_idx_entrenaiento = min(df_actual['index'])
    max_idx_entrenamiento = min_idx_entrenaiento + row['entrenamiento']

    min_idx_validacion = max_idx_entrenamiento
    max_idx_validacion = min_idx_validacion + row['validacion']

    min_idx_prueba = max_idx_validacion
    max_idx_prueba = min_idx_prueba + row['prueba']
    
    tipo_conjunto_1 = 'ENTRENAMIENTO'
    tipo_conjunto_2 = 'VALIDACION'
    tipo_conjunto_3 = 'PRUEBA'    
    
    cond_idx_entrenamiento = ((df_actual.index1 >=  min_idx_entrenaiento) & (df_actual.index1 <  max_idx_entrenamiento))
    cond_idx_validacion = ((df_actual.index1 >=  min_idx_validacion)   & (df_actual.index1 <  max_idx_validacion))
    cond_idx_prueba = ((df_actual.index1 >=  min_idx_prueba)       & (df_actual.index1 <  max_idx_prueba))

    # agregar tipo de conjunto para las imagenes.
    df_actual['tipo_conjunto'] = np.select(
    [
        cond_idx_entrenamiento,
        cond_idx_validacion,
        cond_idx_prueba
    ], 
    [
       tipo_conjunto_1,
       tipo_conjunto_2,
       tipo_conjunto_3
    ], 
    default='_sin_conjunto' )
    
    # agregar ruta de almacenamiento para conjunto de datos de entrenamiento, validacion y pruebas
    root_path_conjunto_pre_proc = 'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO'
    df_actual['path_conjunto_pre_procesamiento'] = np.select(
    [
        cond_idx_entrenamiento,
        cond_idx_validacion,
        cond_idx_prueba
    ], 
    [
        root_path_conjunto_pre_proc + '\\' + tipo_conjunto_1 + '\\PROCESADO\\',
        root_path_conjunto_pre_proc + '\\' + tipo_conjunto_2 + '\\PROCESADO\\',
        root_path_conjunto_pre_proc + '\\' + tipo_conjunto_3 + '\\PROCESADO\\'
    ], 
    default='_sin_conjunto')
    
    # agregar ruta de almacenamiento para conjunto de datos de entrenamiento, validacion y pruebas
    root_path_conjunto_deteccion = 'D:\ADNI - IDA\ADNI1_Complete 1Yr 1.5T\ADNI-PRE-PROCESAMIENTO'
    df_actual['path_conjunto_deteccion'] = np.select(
    [
        cond_idx_entrenamiento,
        cond_idx_validacion,
        cond_idx_prueba
    ], 
    [
        root_path_conjunto_deteccion + '\\' + tipo_conjunto_1 + '\\' + row['clasificacion'] + '\\',
        root_path_conjunto_deteccion + '\\' + tipo_conjunto_2 + '\\' + row['clasificacion'] + '\\',
        root_path_conjunto_deteccion + '\\' + tipo_conjunto_3 + '\\' + row['clasificacion'] + '\\'        
    ], 
    default='_sin_conjunto')
    
    # df_actual =  df_actual.rename(columns={"image_data_id": "image_data_id_x", "subject": "subject_x"})   
    result_dataframe = result_dataframe.append(df_actual, ignore_index=True)

result_dataframe.drop(['index', 'level_0', 'index1'], axis = 1, inplace = True) 
print(result_dataframe.shape)

In [ ]:
result_dataframe.sort_values(by=["orden"], ascending=False, inplace = True)
result_dataframe.tail(3)

## Carga datos de pre procesamiento

In [88]:
for index, row in result_dataframe.iterrows():
    
#     print(row['tipo_conjunto'])
#     print(row['path_jpg'])
#     print(row['path_conjunto_pre_procesamiento'])
#     print(row['path_conjunto_deteccion'])
#     print(row['file_name_jpg'], '\n')
    
    original = row['path_jpg']  + '\\' + row['file_name_jpg'] + '.jpg'
    target_path   = row['path_conjunto_pre_procesamiento'] 
    target   = row['path_conjunto_pre_procesamiento'] + row['file_name_jpg'] + '.jpg'
    
#     print(original, '\n')
#     print(target, '\n')
    
    try:
        if not os.path.exists(target_path):
            os.makedirs(target_path)
    
        os.chmod(target_path, 0o777)
        shutil.copyfile(original, target)
    except:
        print("Error copiando: ", original)

## Definir modelo

In [ ]:
# Numero de iteraciones para los datos de entrenamiento
epochs = 15 

# Número de parches de imágenes usados para calcular un solo
batch_size = 32        

# Tasa de aprendizaje
learning_rate = 0.0001  

# Funcion de optimizacion para los parametros mediante la tasa o gradiente 
optimizer = keras.optimizers.Adam(lr=learning_rate)

# metricas de monitoreo
progress_example = 2
buffer = 128
progress_ims = []


def train(loss_function, model):
    
    model.compile(optimizer=optimizer, loss=loss_function)

    # As the training progresses, we'll monitor network output and performance
    # metrics. Some related variables are initialized here
    example_input = test_input[[3], ...]
    edge_buffer = 128
    progress_ims = []
    progress_val = []

    for epoch in range(epochs): 
        
        
        # Evaluate model on reserved data
        val_loss = model.evaluate(val_input, val_target)
        example_output = model.predict(example_input)
        example_img = example_output[0, edge_buffer:-edge_buffer,
                                  edge_buffer:-edge_buffer, 0]
        progress_ims.append(example_img)
        progress_val.append(val_loss)
        # Update model weights using training data
        istart = 0
        while istart < (len(train_input) - batch_size):
            x = train_input[istart:istart + batch_size]
            y = train_target[istart:istart + batch_size]
            model.train_on_batch(x=x, y=y)
            istart += batch_size

    progress_ims = np.stack(progress_ims, axis=0)

    print('Training phase complete.')
    return model, progress_ims, progress_val


# Now run the training fuction to obtain the trained model and performance at
# intermediate steps
denoising_model, progress_ims, progress_val = train(loss_function='mse', model=denoising_model)

https://pubs.rsna.org/doi/full/10.1148/ryai.2020200036

